# Week 9: Stable Diffusion 종합 실습

이 노트북은 Stable Diffusion을 사용한 Text-to-Image 생성의 전체 실습을 포함합니다.

**실습 내용**:
1. 기본 Diffusion 모델
2. 프롬프트 엔지니어링
3. 스케줄러 비교
4. ControlNet

**권장 환경**: Google Colab (GPU: T4)

---

## 🔧 1. 환경 설정

In [ ]:
# 패키지 설치
!pip install -q diffusers transformers accelerate torch matplotlib opencv-python

In [ ]:
# 라이브러리 임포트
import torch
from diffusers import (
    StableDiffusionPipeline,
    DDIMScheduler,
    DPMSolverMultistepScheduler,
    EulerDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    StableDiffusionControlNetPipeline,
    ControlNetModel
)
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import time

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🚀 사용 디바이스: {device}")
if torch.cuda.is_available():
    print(f"📊 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 🎨 2. 기본 이미지 생성

In [ ]:
# 모델 로드
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16
)
pipe = pipe.to(device)

print("✅ 모델 로드 완료!")

In [ ]:
# 기본 이미지 생성
prompt = "A beautiful sunset over mountains, digital art"
negative_prompt = "low quality, blurry, ugly"

generator = torch.Generator(device).manual_seed(42)

image = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=25,
    guidance_scale=7.5,
    generator=generator,
    height=512,
    width=512
).images[0]

plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title("기본 생성 결과")
plt.axis('off')
plt.show()

## ✍️ 3. 프롬프트 엔지니어링

In [ ]:
# 프롬프트 템플릿 정의
prompts = {
    "realistic": '''
        Portrait of a wise old wizard,
        cinematic photography,
        dramatic lighting with rim light,
        close-up shot,
        ultra-detailed, 8k, sharp focus,
        professional photography
    ''',

    "artistic": '''
        Portrait of a wise old wizard,
        oil painting style,
        soft warm lighting,
        classical composition,
        highly detailed brushwork,
        masterpiece
    ''',

    "fantasy": '''
        Portrait of a wise old wizard,
        fantasy art style,
        magical glowing effects,
        mystical atmosphere,
        trending on artstation,
        digital painting
    '''
}

negative_prompt = '''
    low quality, blurry, artifacts,
    deformed hands, bad anatomy,
    poorly drawn face, ugly,
    watermark, text
'''

In [ ]:
# 이미지 생성 함수
def generate_image(prompt, negative, seed=42):
    generator = torch.Generator(device).manual_seed(seed)

    image = pipe(
        prompt=prompt,
        negative_prompt=negative,
        num_inference_steps=30,
        guidance_scale=7.5,
        generator=generator,
        height=512,
        width=512
    ).images[0]

    return image

In [ ]:
# 여러 스타일 비교
print("🎨 스타일별 이미지 생성 중...")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for idx, (style, prompt) in enumerate(prompts.items()):
    print(f"  - {style}...")
    image = generate_image(prompt, negative_prompt)

    axes[idx].imshow(image)
    axes[idx].set_title(f"{style.upper()}", fontsize=14)
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('prompt_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ 스타일 비교 완료!")

In [ ]:
# 네거티브 프롬프트 효과 비교
print("🔍 네거티브 프롬프트 효과 분석 중...")

fig, axes = plt.subplots(1, 2, figsize=(10, 5))

# 네거티브 없음
image_no_neg = generate_image(prompts["realistic"], "", seed=42)
axes[0].imshow(image_no_neg)
axes[0].set_title("네거티브 프롬프트 없음")
axes[0].axis('off')

# 네거티브 있음
image_with_neg = generate_image(prompts["realistic"], negative_prompt, seed=42)
axes[1].imshow(image_with_neg)
axes[1].set_title("네거티브 프롬프트 적용")
axes[1].axis('off')

plt.tight_layout()
plt.savefig('negative_prompt_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ 네거티브 프롬프트 비교 완료!")

## ⏱️ 4. 스케줄러 비교

In [ ]:
# 스케줄러 딕셔너리
schedulers = {
    'DDIM': DDIMScheduler.from_config(pipe.scheduler.config),
    'DPM-Solver++': DPMSolverMultistepScheduler.from_config(pipe.scheduler.config),
    'Euler': EulerDiscreteScheduler.from_config(pipe.scheduler.config),
    'Euler A': EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)
}

# 테스트 프롬프트
test_prompt = '''
A serene mountain landscape at sunset,
golden hour lighting,
photorealistic style,
ultra-detailed, 8k resolution
'''

test_negative = '''
low quality, blurry, artifacts,
distorted, ugly
'''

In [ ]:
# 스케줄러별 생성 함수
def generate_with_scheduler(scheduler_name, scheduler, steps=25):
    pipe.scheduler = scheduler
    generator = torch.Generator(device).manual_seed(42)

    start_time = time.time()

    image = pipe(
        prompt=test_prompt,
        negative_prompt=test_negative,
        num_inference_steps=steps,
        guidance_scale=7.5,
        generator=generator,
        height=512,
        width=512
    ).images[0]

    elapsed_time = time.time() - start_time

    return image, elapsed_time

In [ ]:
# 스케줄러 비교 실험
print("⏱️ 스케줄러 비교 실험 시작...")
print("=" * 50)

results = {}
fig, axes = plt.subplots(2, 2, figsize=(12, 12))
axes = axes.flatten()

for idx, (name, scheduler) in enumerate(schedulers.items()):
    print(f"\n{idx+1}. {name} 생성 중...")

    image, elapsed = generate_with_scheduler(name, scheduler)
    results[name] = {'image': image, 'time': elapsed}

    # 시각화
    axes[idx].imshow(image)
    axes[idx].set_title(f"{name}\n생성 시간: {elapsed:.2f}초", fontsize=12)
    axes[idx].axis('off')

    print(f"  ✅ {name}: {elapsed:.2f}초")

plt.tight_layout()
plt.savefig('scheduler_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n" + "=" * 50)
print("✅ 스케줄러 비교 완료!")

# 결과 요약
print("\n📊 결과 요약:")
for name, result in results.items():
    print(f"  - {name:15s}: {result['time']:.2f}초")

## 🧩 5. ControlNet (Canny Edge)

In [ ]:
# ControlNet 모델 로드
print("🔧 ControlNet 모델 로딩 중...")

controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny",
    torch_dtype=torch.float16
)

cn_pipe = StableDiffusionControlNetPipeline.from_pretrained(
    model_id,
    controlnet=controlnet,
    torch_dtype=torch.float16
)
cn_pipe = cn_pipe.to(device)

print("✅ ControlNet 로드 완료!")

In [ ]:
# Canny edge 추출 함수
def get_canny_edge(image, low_threshold=100, high_threshold=200):
    image_np = np.array(image)
    if len(image_np.shape) == 3:
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)

    edges = cv2.Canny(image_np, low_threshold, high_threshold)
    edges = cv2.cvtColor(edges, cv2.COLOR_GRAY2RGB)

    return Image.fromarray(edges)

In [ ]:
# 샘플 이미지 로드 및 처리
from urllib.request import urlopen

print("📥 샘플 이미지 다운로드 중...")
url = "https://huggingface.co/lllyasviel/sd-controlnet-canny/resolve/main/images/bird.png"
image = Image.open(urlopen(url))

# Canny edge 추출
canny_image = get_canny_edge(image)

print("✅ 이미지 준비 완료!")

In [ ]:
# ControlNet으로 이미지 생성
print("🎨 ControlNet 이미지 생성 중...")

cn_prompt = "a beautiful bird, digital art style, colorful, detailed"
cn_negative = "low quality, blurry, ugly"

cn_output = cn_pipe(
    prompt=cn_prompt,
    negative_prompt=cn_negative,
    image=canny_image,
    num_inference_steps=30,
    guidance_scale=7.5,
    controlnet_conditioning_scale=1.0
).images[0]

# 결과 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(image)
axes[0].set_title("Original")
axes[0].axis('off')

axes[1].imshow(canny_image)
axes[1].set_title("Canny Edge")
axes[1].axis('off')

axes[2].imshow(cn_output)
axes[2].set_title("ControlNet Output")
axes[2].axis('off')

plt.tight_layout()
plt.savefig('controlnet_result.png', dpi=300)
plt.show()

print("\n✅ ControlNet 생성 완료!")

## 📊 6. 종합 결과

In [ ]:
print("="*50)
print("✅ Week 9 실습 완료!")
print("="*50)
print("\n📁 생성된 파일:")
print("  - prompt_comparison.png")
print("  - negative_prompt_comparison.png")
print("  - scheduler_comparison.png")
print("  - controlnet_result.png")
print("\n💡 다음 단계:")
print("  1. 다양한 프롬프트 실험")
print("  2. 파라미터 최적화")
print("  3. LoRA 모델 적용")
print("  4. ComfyUI 워크플로우 구성")